In [69]:
import json
from pprint import pprint
import os
import glob
import re
import imp
import sys
import inspect

In [109]:
proj_dir = os.path.expanduser('/home/dimert/repos/vacancy_analysis/')

def do_skip_dir(dirname):
    exclude_dirs = ['ropeproject', 'tests']
    skipdir = False
    for excl_dir in exclude_dirs:
        if re.search(excl_dir, dirname):
            skipdir = True
            break
    return skipdir

def python_files(proj_dir):
    for dirpath, _, files in os.walk(proj_dir):
        if do_skip_dir(dirpath):
            continue
        for fname in files:
            if re.search(r'\.py$', fname) and not re.search(r'setup\.py', fname):
                yield os.path.abspath(os.path.join(dirpath, fname))
                
def get_methods(cls):   
  return [i for i in cls.__dict__.keys() if i[:2] != '__' and inspect.isfunction(cls.__dict__[i])]

def get_attrs(cls):   
  return [i for i in cls.__dict__.keys() if i[:2] != '__' and not inspect.isfunction(cls.__dict__[i])]

def get_classes_from_file(fname):
    classes = {}
    try:
        module = imp.load_source('', fname)
    except:
        return classes
    for clsname, cls in inspect.getmembers(module, inspect.isclass):
        classes[clsname] = {'attrs': get_attrs(cls), 'methods': get_methods(cls)}
    return classes

def get_classes(proj_dir):
    sys.path.append(proj_dir)
    all_classes = {}
    for fname in python_files(proj_dir):
        #print(pfile)
        classes = get_classes_from_file(fname)
        all_classes.update(classes)
    from pprint import pprint
    pprint(all_classes)
    
#get_classes_from_file()
x = imp.load_source('/home/dimert/NMS/nms/core/power_control.py', '/home/dimert/NMS/nms/core/power_control.py')
for name, obj in inspect.getmembers(x):
    if inspect.isclass(obj):
        if obj.__module__ == '/home/dimert/NMS/nms/core/power_control.py':
            print(obj)

<class '/home/dimert/NMS/nms/core/power_control.py.GlobalPowerManager'>
<class '/home/dimert/NMS/nms/core/power_control.py.PDUController'>
<class '/home/dimert/NMS/nms/core/power_control.py.ServerPowerManager'>


In [124]:
x = '        self.command_123 = ""\n'
print(x)
par_name = re.findall(r'self\.(.+)\s=', x)
par_name

        self.command_123 = ""



['command_123']